In [1]:
from auth import get_creds
from model import GoogleDrive, DriveFile
from pathlib import Path

In [2]:
creds, _ = get_creds('royce.melborn2@gmail.com')
drive = self = GoogleDrive(creds)
file_list = drive.list_all_files()

In [3]:
drive = GoogleDrive(creds)
root = DriveFile(drive.get_root())
root.path = Path('.')

drive.generate_tree(root)


In [8]:
root.path

PosixPath('.')

In [9]:
data_path = Path.home() / ".drive_sync"

config_path = data_path / "config.yaml"

In [13]:
import yaml
with config_path.open() as f:
    config = yaml.safe_load(f)
config

{'gdrive': {'royce.melborn2@gmail.com': {'target': '/home/royce/rdrive'},
  'gnadh2007@gmail.com': {'target': '/home/royce/ggdrive'}}}

In [6]:
f = DriveFile(file_list[1])
f.generate_paths(base_path='/home/royce/Downloads/drive')
self.download(f)

In [5]:
self.download('1ni4AbinoRy0s-9eECJZJd8LRRizvwVQA', Path('/home/royce/Downloads/geetha'))

In [18]:
from enum import Enum

class MimeType(Enum):
    FOLDER = "application/vnd.google-apps.folder"
    GSHEET = "application/vnd.google-apps.spreadsheet"
    OTHER = 'unknown'

    @classmethod
    def _missing_(cls, value):
        return MimeType.OTHER

mime = MimeType('pplication/vnd.google-apps.spreadsheet')

mime

<MimeType.OTHER: 'unknown'>